# Análisis de texto al portal web del periodico pagina12

En este proyecto seccionado en dos fases se pretende realizar un análisis de texto a las noticias que se encuentran en el protal web del periodico argentino pagina12, con el fin de determinar el tipo de noticias mas recurrentes basado en el tipo de palabras y expresiones mas  empleadas en sus editoriales, intentando de esta manera comprender cual es la linea o filosofia del periodico y su posible orientación politica. El proyecto se desarrollara en dos fases:
1. Se realizara un  proceso de web scraping que permita obtener de manera automatica información de las noticias presentes en sus distintas secciones, para luego construir un dataframe con la información principal.

2. se realizara un  proceso de *text mining* donde se analizaran patrones , tendencias, wordcloud y se construira en última instancia un perfil del tipo de noticias y el enfoque dado por el periodico a las mismas(por ejemplo ver si ante noticias que rebatan el desempeño del gobierno es: más critico, tolerante o si  siempre refleja un apoyo al estado  ).


In [2]:
# importamos los paquetes a usar

import requests
from bs4 import BeautifulSoup as bs
from IPython.display import display, Image

In [3]:
# definamos la url a la que queremos scrappear
url = 'https://www.pagina12.com.ar/'

In [4]:
# ahora vamos a hacer la solicitud o requests al sitio web pagina 12
p12 = requests.get(url)

In [5]:
# el siguiente codigo nos indica si el sitio nos da response
p12.status_code

200

In [6]:
p12.headers

{'Date': 'Mon, 16 Dec 2019 18:53:24 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=d8af3caba6131f615104de7af9647bdf91576522404; expires=Wed, 15-Jan-20 18:53:24 GMT; path=/; domain=.pagina12.com.ar; HttpOnly', 'Vary': 'Accept-Encoding', 'X-DNS-Prefetch-Control': 'off', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Download-Options': 'noopen', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'X-Backend': 'prod_frontend_1', 'X-Backend-TTL': '180.000', 'X-Type': 'Dynamic URI', 'Age': '0', 'grace': '86400.000 none', 'ttl': '119.806', 'x-debug': '', 'X-Instance': 'cache-front-prod-varnish-76c5f88cf6-264lk', 'x-restarts': '0', 'X-Cache': 'HIT (1)', 'CF-Cache-Status': 'DYNAMIC', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'cloudflare', 'CF-RAY': '5462dc215db7e5c8-BOG', 'Content-Encoding': '

In [7]:
# luego vamos a obtener un objeto Beautifulsoup para poder parsear la pagina
pag_princ = bs(p12.text, 'lxml')
# para ver la estructura interna de la url
print(pag_princ.prettify())

<!DOCTYPE html>
<html class="no-js">
 <head>
  <meta charset="utf-8"/>
  <title>
   Página12 | La otra mirada
  </title>
  <meta content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg" name="google-site-verification"/>
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "Organization","url": "https://www.pagina12.com.ar","logo": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png"}
  </script>
  <meta content="Notas focalizadas en el quehacer político de la Argentina." name="description" property="description"/>
  <meta content="1541638399393436" name="fb:pages" property="fb:pages"/>
  <meta content="es_AR" name="og:locale" property="og:locale"/>
  <meta content="Página12 | La otra mirada" name="og:title" property="og:title"/>
  <meta content="PAGINA12" name="og:site_name" property="og:site_name"/>
  <meta content="website" name="og:type" property="og:type"/>
  <meta content="https://www.pagina12.com.ar" name="og:url" property="og:url"/>
  <meta 

In [8]:
# analizando el contenido html de la pagina vemos que para obtener la barra principal 
#de contenidos de la pagina necesitamos:

secciones = pag_princ.find('ul', attrs = {'class':'hot-sections'})
secciones = secciones.find_all('li')
secciones = [ seccion.a.get('href') for seccion in secciones]
secciones



['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/dialogos',
 'https://www.pagina12.com.ar/secciones/el-mundo',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/contratapa']

In [9]:
# vamos a analizar una sección, a la cual le vamos a extraer su info interna que viene estructurada como:
# 1. noticia principal
# 2. noticias secundarias
# tomemos la primera seccion(PAIS)
seccion = secciones[0]
seccion

'https://www.pagina12.com.ar/secciones/el-pais'

In [10]:
# hacemos el mismo procedimiento para esta sección
sec = requests.get(seccion)
sec.status_code


200

In [11]:
secc = bs(sec.text, 'lxml')
print(secc.prettify())

<!DOCTYPE html>
<html class="no-js">
 <head>
  <meta charset="utf-8"/>
  <title>
   El país | Página12
  </title>
  <link href="https://www.pagina12.com.ar/secciones/el-pais" rel="canonical"/>
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "Organization","url": "https://www.pagina12.com.ar","logo": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png"}
  </script>
  <meta content="La puja política de coyuntura y la reflexión de prestigiosos columnistas sobre la actualidad.
" name="description" property="description"/>
  <meta content="1541638399393436" name="fb:pages" property="fb:pages"/>
  <meta content="es_AR" name="og:locale" property="og:locale"/>
  <meta content="El país | Página12" name="og:title" property="og:title"/>
  <meta content="PAGINA12" name="og:site_name" property="og:site_name"/>
  <meta content="website" name="og:type" property="og:type"/>
  <meta content="https://www.pagina12.com.ar/secciones/el-pais" name="og:url" prope

In [12]:
# ahora quiero tomar el link de la noticia principal y las noticias secundarias
# para ello creamos una funciñon que facilmente nos acceda a esta info.

def seccion_noticias(BS):
    noticias = []
    noticia_princ =  BS.find('div', attrs={'class':'featured-article__container'})
    noticia_princ = noticia_princ.a.get('href')
    noticias.append(noticia_princ)
    noticia_secund = BS.find('ul', attrs={'class':'article-list'})
    noticia_secund = noticia_secund.find_all('h2')
    links_noti_secund= [ elemento.a.get('href') for elemento in noticia_secund]
    for articulo in  links_noti_secund:
        noticias.append(articulo)
    return noticias
    

In [13]:
print(seccion_noticias(secc))

['https://www.pagina12.com.ar/236849-churrasco-sandoval-llego-a-la-argentina-para-ser-enjuiciado', 'https://www.pagina12.com.ar/236838-evo-morales-agradecio-a-los-hermanos-alberto-y-cristina', 'https://www.pagina12.com.ar/236834-paro-sorpresivo-de-colectivos', 'https://www.pagina12.com.ar/236831-el-campo-que-apoya-las-retenciones', 'https://www.pagina12.com.ar/236821-daniel-arroyo-se-reunio-con-los-ministros-provinciales-del-a', 'https://www.pagina12.com.ar/236818-desvalijaron-la-casa-de-dujovne-en-punta-del-este', 'https://www.pagina12.com.ar/236816-la-ley-de-emergencia-economica-llega-al-congreso', 'https://www.pagina12.com.ar/236814-alberto-fernandez-suspendera-el-pacto-fiscal', 'https://www.pagina12.com.ar/236710-lo-que-no-ofrece-dudas-sin-modelo-perverso-en-su-origen', 'https://www.pagina12.com.ar/236717-alberto-fernandez-no-quiere-guerra-con-el-campo', 'https://www.pagina12.com.ar/236726-asuncion-de-fernandez-como-fue-la-visita-del-presidente-de-c', 'https://www.pagina12.com.ar/2

In [14]:
# tememos ahora una de estas y obtengamos su titulo, autor, fecha de pub y cuerpo de la noticia
sub_sec = seccion_noticias(secc)
#vamos a crear un bloque try-except que nos permita informarnos de posibles errores en el proceso

try:
    url_sub = sub_sec[0]
    sub_not = requests.get(url_sub)
    if sub_not.status_code == 200:
        sub_noticia = bs(sub_not.text,'lxml')
        # extraemos el tituto de la noticia
        titulo = sub_noticia.find('div', attrs={'class':'article-title'})
        print('Titulo \n', titulo.text)
    # extraemos la fecha del articulo 
        fecha = sub_noticia.find('span', attrs = {'pubdate':'pubdate'}).get('datetime')
        print('Fecha publicación \n', fecha)
    # extraemos la volanta:
        try:
            volanta = sub_noticia.find('div', attrs={'class':'article-prefix'})
            print('Volanta\n', volanta.text)
        except:
            print('Volanta:\n', 'no hay volanta')
    # cuerpo de la noticia
        cuerpo = sub_noticia.find('div', attrs= {'class':'article-text'})
        print('Cuerpo\n\n',cuerpo.text, '\n')
    # extraemos el resumen
        try:
            resumen = sub_noticia.find('div', attrs= {'class':article-summary})
            print('Resumen:\n', resumen)
        except:
            print('Resumen:\n','no hay resumen')
    # extraemos el autor 
        try:
            autor = sub_noticia.find('div', attrs={'class':'article-author'}).span.a.get_text()
            print('Autor:\n', autor)
        except:
            print('Autor:', 'No hay autor')
        
except Exception as e:
        print('Error:')
        print(e)
        print('\n')
        
    

Titulo 
 "Churrasco" Sandoval llegó a la Argentina para ser enjuiciado
Fecha publicación 
 2019-12-16
Volanta
 El represor fue extraditado desde Francia tras siete años de dilaciones
Cuerpo

 El ex policía Mario Sandoval, acusado por la desaparición de un estudiante durante la última dictadura militar, llegó a las 10.09 al aeropuerto de Ezeiza en un vuelo de Air France, tras ser extraditado desde Francia. De inmediado fue trasladado a Comodoro Py para ser indagado.Sandoval fue extraditado desde Francia, luego de que el Tribunal Europeo de Derechos Humanos rechazara la semana pasada un recurso presentado por el acusado para suspender la medida, reclamada por la Justicia argentina desde 2012.El juez federal Sergio Torres había solicitado su extradición en el marco de una causa que investiga unos 800 casos de desapariciones en la exEscuela de Mecánica de la Armada (ESMA), el mayor centro clandestino de detención durante la dictadura por el que se estima pasaron unos 5.000 prisioneros.El e

In [15]:
# ahora vamos a descargar la imagen que acompaña la noticia
def imagenes(BS):
    try:
        media = BS.find('div', attrs={'class':'article-main-media-image'})
        imagenes = media.find_all('img')
        if len(imagenes)==0:
            print('No hay imagenes')
        else:
        # quiero quedarme con la imagen mas grande
            imagen = imagenes[-1]
        # tomamos solo la src(source)
            img_src = imagen.get('data-src')
        # ahora hagamos la request de la imagen
            img_req = requests.get(img_src)
            if img_req.status_code ==200:
                pic = img_req.content
            else:
                print('No se puedo obtener la imagen')
                
            
    except Exception as e:
        print('error')
        print(e)
    return pic


In [17]:
# ahora creamos un diccionario que retenga la info de la pagina
def obtener_info(s_nota):
    # partimos creando un diccionario vacio donde agregaremos los campos de interes
    get_info = {}
    try:
        # iniciemos obteniendo el titulo de la noticia
        try:
            titulo = s_nota.find('div', attrs={'class':'article-title'}) 
            get_info['titulo']=titulo.get_text()
        except:
            get_info['titulo'] = None
        
        # vamos a obtener la fecha
        try:
            fecha = s_nota.find('span', attrs = {'pubdate':'pubdate'}).get('datetime')
            get_info['fecha'] = fecha
        except:
            get_info['fecha'] = None
        # extraemos el autor de la noticia
        try:
            autor = s_nota.find('div', attrs={'class':'article-author'}).a.get_text()
            get_info['autor'] = autor
        except:
            get_info['autor'] = None
        
        # ahora vamos a obtener el titular adicional o volanta 
        try:
            volanta = s_nota.find('div', attrs={'class':'article-prefix'})
            get_info['volanta'] = volanta.get_text()
        except:
            get_info['volanta'] = None
            
        # obtengamos a continuacion el cuerpo de la noticia
        try:
            cuerpo = s_nota.find('div', attrs= {'class':'article-text'})
            get_info['cuerpo'] = cuerpo.get_text() 
        except:
            get_info['cuerpo'] = None
        # ahora extraemos el resumen de la noticia
        try:
            resumen = s_nota.find('div', attrs= {'class':article-summary})
            get_info['resumen'] = resumen.get_text()
        except:
            get_info['resumen']= None  
            
           # extraemos finalmente la imagen principal que acompaña el articulo usando
           # la funcion imagenes que previmante construimos
        try:
            get_info['imagen'] = imagenes(s_nota)
        except:
            get_info['imagen'] = None
        
    except:
        print('No es posible acceder a la sección')
    return get_info
    
    


In [18]:

#vamos a crear una funciòn a la que se le ingrese la url y nos haga el resquest 
def solicitud_request(url):
    try:
        nota = requests.get(url)
    except Exception as e:
        print('No se puedo acceder a la url', url)
        print(e)
        return None
    if nota.status_code!=200:
        print('Error al tratar de obtener la url', url)
        print('nota.status_code', nota.status_code)
        return None
    s_nota = bs(nota.text, 'lxml')
    info_nota = obtener_info(s_nota)
    info_nota['url'] = url
    return info_nota

    

In [19]:
solicitud_request(sub_sec[0])

{'titulo': '"Churrasco" Sandoval llegó a la Argentina para ser enjuiciado',
 'fecha': '2019-12-16',
 'autor': 'Florencia Barragan',
 'volanta': 'El represor fue extraditado desde Francia tras siete años de dilaciones',
 'cuerpo': 'El ex policía Mario Sandoval, acusado por la desaparición de un estudiante durante la última dictadura militar, llegó a las 10.09 al aeropuerto de Ezeiza en un vuelo de Air France, tras ser extraditado desde Francia. De inmediado fue trasladado a Comodoro Py para ser indagado.Sandoval fue extraditado desde Francia, luego de que el Tribunal Europeo de Derechos Humanos rechazara la semana pasada un recurso presentado por el acusado para suspender la medida, reclamada por la Justicia argentina desde 2012.El juez federal Sergio Torres había solicitado su extradición en el marco de una causa que investiga unos 800 casos de desapariciones en la exEscuela de Mecánica de la Armada (ESMA), el mayor centro clandestino de detención durante la dictadura por el que se est

In [20]:
notas = []
for seccion in secciones:
    try:
        consulta_URL = requests.get(seccion)
        if consulta_URL.status_code ==200:
            sec_soup = bs(consulta_URL.text, 'lxml')
            notas.extend(seccion_noticias(sec_soup))
        else:
            print('No se pudo obtener la sección', seccion)
    except:
        print('No se puedo acceder a la seccion', seccion)
            
            

No se puedo acceder a la seccion https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos


In [21]:
notas

['https://www.pagina12.com.ar/236849-churrasco-sandoval-llego-a-la-argentina-para-ser-enjuiciado',
 'https://www.pagina12.com.ar/236838-evo-morales-agradecio-a-los-hermanos-alberto-y-cristina',
 'https://www.pagina12.com.ar/236834-paro-sorpresivo-de-colectivos',
 'https://www.pagina12.com.ar/236831-el-campo-que-apoya-las-retenciones',
 'https://www.pagina12.com.ar/236821-daniel-arroyo-se-reunio-con-los-ministros-provinciales-del-a',
 'https://www.pagina12.com.ar/236818-desvalijaron-la-casa-de-dujovne-en-punta-del-este',
 'https://www.pagina12.com.ar/236816-la-ley-de-emergencia-economica-llega-al-congreso',
 'https://www.pagina12.com.ar/236814-alberto-fernandez-suspendera-el-pacto-fiscal',
 'https://www.pagina12.com.ar/236710-lo-que-no-ofrece-dudas-sin-modelo-perverso-en-su-origen',
 'https://www.pagina12.com.ar/236717-alberto-fernandez-no-quiere-guerra-con-el-campo',
 'https://www.pagina12.com.ar/236726-asuncion-de-fernandez-como-fue-la-visita-del-presidente-de-c',
 'https://www.pagina

In [22]:
# Ahora voy a tomar cada uno de os links de las notas y voy a aplicar la funcion solicitud_request
# es claro que al aplicar esta función estoy 'scrapreando' o arrastarndo la info
# de cada una de las noticias que aparecen al entrar en cada sección del periodico 
datos = []
for i, nota in enumerate(notas):
    print(f'Scrapeando la nota{i}/{len(notas)}')
    if solicitud_request(nota)==None:
        print('no se agregará  la lista por problemas de ulr', nota)
    else:
        datos.append(solicitud_request(nota))
    

Scrapeando la nota0/101
Scrapeando la nota1/101
Scrapeando la nota2/101
Scrapeando la nota3/101
Scrapeando la nota4/101
Scrapeando la nota5/101
Scrapeando la nota6/101
Scrapeando la nota7/101
Scrapeando la nota8/101
Scrapeando la nota9/101
Scrapeando la nota10/101
Scrapeando la nota11/101
Scrapeando la nota12/101
Scrapeando la nota13/101
Scrapeando la nota14/101
Scrapeando la nota15/101
Scrapeando la nota16/101
Scrapeando la nota17/101
Scrapeando la nota18/101
Scrapeando la nota19/101
Scrapeando la nota20/101
Scrapeando la nota21/101
Scrapeando la nota22/101
Scrapeando la nota23/101
Scrapeando la nota24/101
Scrapeando la nota25/101
Scrapeando la nota26/101
Scrapeando la nota27/101
Scrapeando la nota28/101
Scrapeando la nota29/101
Scrapeando la nota30/101
Scrapeando la nota31/101
Scrapeando la nota32/101
Scrapeando la nota33/101
Scrapeando la nota34/101
Scrapeando la nota35/101
Scrapeando la nota36/101
Scrapeando la nota37/101
error
'NoneType' object has no attribute 'find_all'
error
'N

In [26]:
len(datos)

101

In [24]:
# finalmente creamos un data frame con la info obtenida
import pandas as pd
df = pd.DataFrame(datos[])

In [25]:
df.head()

,titulo,fecha,autor,volanta,cuerpo,resumen,imagen,url
0,"""Churrasco"" Sandoval llegó a la Argentina para...",2019-12-16,Florencia Barragan,El represor fue extraditado desde Francia tras...,"El ex policía Mario Sandoval, acusado por la d...",None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.pagina12.com.ar/236849-churrasco-s...
1,"Evo Morales agradeció a ""los hermanos Alberto ...",2019-12-16,Florencia Barragan,Por recibirlo como refugiado político,"El expresidente de Bolivia Evo Morales\n, que ...",None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.pagina12.com.ar/236838-evo-morales...
2,Paro sorpresivo de colectivos,2019-12-16,Florencia Barragan,Más de 40 líneas afectadas por un reclamo de l...,Los trabajadores de la empresa Dota convocaron...,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.pagina12.com.ar/236834-paro-sorpre...
3,El campo que apoya las retenciones,2019-12-16,Florencia Barragan,El Foro Agrario Nacional respaldó la medida de...,"El Foro Agrario Nacional, que reúne a producto...",None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.pagina12.com.ar/236831-el-campo-qu...
4,Daniel Arroyo se reunió con los ministros prov...,2019-12-16,Florencia Barragan,Convocó al encuentro para coordinar políticas ...,Daniel Arroyo se reunió con los ministros prov...,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.pagina12.com.ar/236821-daniel-arro...


In [27]:
# finalmente guardamos la informacion en un archivo csv
df.to_csv('Noticias pagina 12')